In [6]:
import tensorflow as tf
import numpy as np
from operator import mul

from tensorflow.contrib import slim

In [2]:
#refer:https://blog.csdn.net/two_vv/article/details/76769860
#alexnet原始模型以及预训练参数导入
class AlexNet_model(object):
    def __init__(self , is_training=True):
        
        self.x = tf.placeholder(dtype=tf.float32 , shape=[1 , 448 , 448 , 3])
        self.target = tf.placeholder(dtype=tf.float32 , shape=[7 , 7 , 30])
        
        
        #self.build(is_training)
        
            
    

    def build(self , is_training=True , keep_prob=0.5):
        def _conv(_input , num_outputs , kernel_size , stride=1):
            return slim.conv2d(_input , num_outputs=num_outputs , kernel_size=kernel_size , stride=stride , activation_fn=tf.nn.leaky_relu ,
                             weights_initializer=tf.initializers.truncated_normal(stddev=0.01) ,
                             biases_initializer=tf.initializers.constant(0.0))

        def _max_pool(_input , kernel_size=2 , stride=2):
            return slim.max_pool2d(_input , kernel_size=kernel_size , stride=stride)


        def _conv_module_a(_input):
            _output = _conv(_input , 256 , 1)
            return _conv(_output , 512 , 3)

        def _conv_module_b(_input):
            _output = _conv(_input , 512 , 1)
            return _conv(_output , 1024 , 3)

        output = _conv(self.x , 64 , 7 , 2)                
        output = _max_pool(output)

        output = _conv(output , 192 , 3)              
        output = _max_pool(output)

        output = _conv(output , 128 , 1)
        output = _conv(output , 256 , 3)
        output = _conv(output , 256 , 1)
        output = _conv(output , 512 , 3)
        output = _max_pool(output)

        #4 times
        output = _conv_module_a(output)
        output = _conv_module_a(output)
        output = _conv_module_a(output)
        output = _conv_module_a(output)

        output = _conv(output , 512 , 1)
        output = _conv(output , 1024 , 3)
        output = _max_pool(output)

        #twice
        output = _conv_module_b(output)
        output = _conv_module_b(output)

        output = _conv(output , 1024 , 3)
        output = _conv(output , 1024 , 3 , 2)

        output = _conv(output , 1024 , 3)
        output = _conv(output , 1024 , 3)

        output = slim.flatten(output)

        output = slim.fully_connected(inputs=output , num_outputs=4096 , activation_fn=tf.nn.leaky_relu ,
                                     weights_initializer=tf.initializers.truncated_normal(stddev=0.01),
                                     biases_initializer=tf.initializers.constant(0.0))

        output = slim.fully_connected(inputs=output , num_outputs=7*7*30 , activation_fn=tf.identity ,
                                     weights_initializer=tf.initializers.truncated_normal(stddev=0.01),
                                     biases_initializer=tf.initializers.constant(0.0))

        output = tf.reshape(output , shape=[7 , 7 , 30])

        return output
        

In [3]:
test=AlexNet_model()

In [4]:
test.build().get_shape()

TensorShape([Dimension(7), Dimension(7), Dimension(30)])

In [9]:
num_params = 0
for variable in tf.trainable_variables():
    shape = variable.get_shape().as_list()
    num_params+=np.prod(shape)

In [11]:
num_params*4/1024/1024/1024

1.0121745988726616

In [28]:
a=np.array([
            [[0.7],[0.0],[0.2]],
    [[0.0],[0.2],[0.0]],
    [[0.1],[0.0],[0.8]]
        ])

In [30]:
a

array([[[0.7],
        [0. ],
        [0.2]],

       [[0. ],
        [0.2],
        [0. ]],

       [[0.1],
        [0. ],
        [0.8]]])

In [31]:
with tf.Session() as sess:
    print( sess.run( tf.cast ( tf.greater(a , np.zeros(shape=[3,3,1] , dtype=float))  , dtype=tf.float32) ) )

[[[1.]
  [0.]
  [1.]]

 [[0.]
  [1.]
  [0.]]

 [[1.]
  [0.]
  [1.]]]


In [52]:
output = tf.placeholder(dtype=tf.float32 , shape=[7 , 7 , 30])
target = tf.placeholder(dtype=tf.float32 , shape=[7 , 7 , 30])

In [53]:
output_slice = tf.slice(output , begin=[0,0,1] , size=[7,7,1])

In [54]:
output_slice.get_shape()

TensorShape([Dimension(7), Dimension(7), Dimension(1)])

In [56]:
output[:,:,[1,5]].get_shape()

TypeError: can only concatenate list (not "int") to list

In [46]:
sub = output[:,:,[1,2]] - target[:,:,[1,2]]

TypeError: can only concatenate list (not "int") to list

In [1]:
import numpy as np 

In [25]:
prod = np.random.normal(size=[98 , 20])


In [26]:
prod

array([[-0.63398765,  0.34829897, -0.78151863, ...,  0.46488947,
        -0.25825126, -0.2039102 ],
       [ 0.5148164 ,  0.73235106, -0.31601091, ..., -0.1308469 ,
        -0.64001386, -0.14228714],
       [ 0.67522538, -1.48651101,  0.60902514, ..., -1.94499995,
         1.04767855, -0.88665212],
       ...,
       [ 1.43564004, -0.39099294,  1.0313145 , ...,  0.16701395,
        -1.1685061 , -1.34797773],
       [ 0.13084994,  1.34226736,  0.87856261, ...,  1.09514145,
         1.40446925, -0.93354471],
       [-1.36354312, -0.58768639, -0.87145072, ...,  0.83688675,
         0.26083658, -0.440279  ]])

In [27]:
#进行阈值mask
prod = np.greater(prod , 0.2).astype(dtype=int) * prod

In [28]:
prod

array([[-0.        ,  0.34829897, -0.        , ...,  0.46488947,
        -0.        , -0.        ],
       [ 0.5148164 ,  0.73235106, -0.        , ..., -0.        ,
        -0.        , -0.        ],
       [ 0.67522538, -0.        ,  0.60902514, ..., -0.        ,
         1.04767855, -0.        ],
       ...,
       [ 1.43564004, -0.        ,  1.0313145 , ...,  0.        ,
        -0.        , -0.        ],
       [ 0.        ,  1.34226736,  0.87856261, ...,  1.09514145,
         1.40446925, -0.        ],
       [-0.        , -0.        , -0.        , ...,  0.83688675,
         0.26083658, -0.        ]])

In [29]:
i=0

In [30]:
np.argsort(prod[i])

array([ 0, 16, 14, 12, 18,  8,  7, 19,  5,  3,  2, 11,  6, 10,  1,  9, 15,
       17, 13,  4], dtype=int64)

In [35]:

np.argsort(-1*np.array([123,12,23,43]))

array([0, 3, 2, 1], dtype=int64)

In [41]:
a=[1,2,3,4]

In [42]:
a[2:]=[6]*(4-2)

In [43]:
a

[1, 2, 6, 6]

In [45]:
a=np.array(a)

In [46]:
a

array([1, 2, 6, 6])

In [47]:
a[2:]=0

In [48]:
a

array([1, 2, 0, 0])